## Analysis on acute lower respiratory infections

Team: Runtime Terrors

Members:    
> Vinu Prasad Bhambore (vpb2)

> Srijith Srinath (ssrina2)

> Dhruman Jayesh Shah (djshah5)

##### Notebook Update 2 - 03/13/2020

Name of the dataset: PAHO Regional Mortality Dataset
    
Background: The PAHO Regional Mortality Database is an integrated database consisting of national datasets from Member States and is updated annually. The dataset includes data from 48 countries and territories.  The source of the national datasets varies by country. For some countries the national institution mandated to collect, integrate, and disseminate mortality data and information is the Ministry of Health, and for others it is the National Institute of Statistics.

In [1]:
import pandas as pd
import numpy as np
import warnings
from collections import defaultdict
import matplotlib.pyplot as plt 
import seaborn as sns 

### Reading the Mortality Dataset

In [2]:
mortality_df = pd.read_csv('Mortality_Data.csv')

In [3]:
mortality_df.head()

,CountryName,MortalityYear,Gender,AgeGroupCode,ICD10,Deaths
0,Brazil,2017,Male,21,I479,1
1,Brazil,2017,Male,21,C925,1
2,Brazil,2017,Male,21,I451,1
3,Brazil,2017,Male,21,D292,1
4,Brazil,2017,Male,21,L519,1


Filtering countries by Bronchities and Tuberculosis

In [4]:
bronchitis_df = mortality_df[mortality_df['ICD10'].str.match(r'(^J20[0-9]*)|(^J40[0-9]*)|(^J41[0-9]*)|(^J42[0-9]*)')].copy()
bronchitis_df['Class'] = 'Bronchitis'
bronchitis_df.head()

,CountryName,MortalityYear,Gender,AgeGroupCode,ICD10,Deaths,Class
563,Brazil,2017,Male,21,J410,1,Bronchitis
885,Brazil,2017,Male,17,J40,1,Bronchitis
2106,Brazil,2017,Male,16,J42,1,Bronchitis
3794,Brazil,2017,Male,22,J418,1,Bronchitis
4444,Brazil,2017,Male,20,J411,1,Bronchitis


In [5]:
tuberculosis_df = mortality_df[mortality_df['ICD10'].str.match(r'(^A15[0-9]*)|(^A17[0-9]*)|(^A18[0-9]*)|(^A19[0-9]*)')].copy()
tuberculosis_df['Class'] = 'Tuberculosis'
tuberculosis_df.head()

,CountryName,MortalityYear,Gender,AgeGroupCode,ICD10,Deaths,Class
515,Brazil,2017,Male,21,A178,1,Tuberculosis
598,Brazil,2017,Male,21,A182,1,Tuberculosis
684,Brazil,2017,Male,21,A180,1,Tuberculosis
858,Brazil,2017,Male,17,A156,1,Tuberculosis
874,Brazil,2017,Male,17,A198,1,Tuberculosis


Combining the bronchities and tuberculosis Dataframes

In [6]:
mortality_2_df = pd.concat([bronchitis_df,tuberculosis_df])
mortality_2_df.reset_index(drop=True, inplace=True)
mortality_2_df.head()

,CountryName,MortalityYear,Gender,AgeGroupCode,ICD10,Deaths,Class
0,Brazil,2017,Male,21,J410,1,Bronchitis
1,Brazil,2017,Male,17,J40,1,Bronchitis
2,Brazil,2017,Male,16,J42,1,Bronchitis
3,Brazil,2017,Male,22,J418,1,Bronchitis
4,Brazil,2017,Male,20,J411,1,Bronchitis


In [7]:
## make changes to the country names, in order to get same formating throughout
mortality_2_df.loc[mortality_2_df['CountryName']=='United States of America', 'CountryName'] = 'United States'
mortality_2_df.loc[mortality_2_df['CountryName']=='Saint Vincent and the Grenadines', 'CountryName'] = 'St. Vincent and the Grenadines'
mortality_2_df.loc[mortality_2_df['CountryName']=='Venezuela', 'CountryName'] = 'Venezuela, RB'
mortality_2_df.loc[mortality_2_df['CountryName']=='Saint Lucia', 'CountryName'] = 'St. Lucia'
mortality_2_df.loc[mortality_2_df['CountryName']=='SaintKittsandNevis', 'CountryName'] = 'St. Kitts and Nevis'
mortality_2_df.loc[mortality_2_df['CountryName']=='Virgin Islands (US)', 'CountryName'] = 'Virgin Islands (U.S.)'
mortality_2_df.loc[mortality_2_df['CountryName']=='TurksandCaicosIslands', 'CountryName'] = 'Turks and Caicos Islands'
mortality_2_df.loc[mortality_2_df['CountryName']=='CaymanIslands', 'CountryName'] = 'Cayman Islands'
mortality_2_df.loc[mortality_2_df['CountryName']=='Brazil ', 'CountryName'] = 'Brazil'
mortality_2_df.loc[mortality_2_df['CountryName']=='Bahamas', 'CountryName'] = 'Bahamas, The'
mortality_2_df.loc[mortality_2_df['CountryName']=='Bolivia ', 'CountryName'] = 'Bolivia'


## remove countries that aren't available in World Bank Datasets
mortality_2_df = mortality_2_df[mortality_2_df['CountryName']!='Montserrat']
mortality_2_df = mortality_2_df[mortality_2_df['CountryName']!='Martinique']
mortality_2_df = mortality_2_df[mortality_2_df['CountryName']!='Guadeloupe']
mortality_2_df = mortality_2_df[mortality_2_df['CountryName']!='French Guiana']


Discovering the total number of countries where the deaths caused were due to bronchitis and tuberculosis

In [8]:
list_of_countries = mortality_2_df['CountryName'].unique()
list_of_countries

array(['Brazil', 'Mexico', 'United States', 'Colombia', 'Argentina',
       'Peru', 'Canada', 'Chile', 'Cuba', 'Guatemala', 'Paraguay',
       'Panama', 'Uruguay', 'Puerto Rico',
       'St. Vincent and the Grenadines', 'Venezuela, RB', 'Nicaragua',
       'St. Lucia', 'El Salvador', 'Ecuador', 'Costa Rica', 'Jamaica',
       'Suriname', 'Belize', 'Aruba', 'St. Kitts and Nevis', 'Dominica',
       'Dominican Republic', 'Honduras', 'Barbados',
       'Antigua and Barbuda', 'Grenada', 'Bolivia', 'Haiti',
       'Trinidad and Tobago', 'Bermuda', 'Guyana', 'Curacao',
       'Bahamas, The', 'Virgin Islands (U.S.)', 'Cayman Islands',
       'Turks and Caicos Islands'], dtype=object)

In [9]:
len(list_of_countries)

42

### Now, we'll look at the Health Expenditure per capita dataset obtained from World Data Bank

Description of the dataset:
The dataset contains data of healt expenditure per capita for each country ranging from the years 1960 - 2016. However, we only have data values from the years 2000 - 2016, hence we will drop the rest of the columns.

##### Reading the dataset

In [10]:
health_df = pd.read_csv('Health_Expenditure.csv')

In [11]:
health_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,45.587750,51.553259,52.218506,55.967550,60.112761,60.088813,57.248760,NaN,NaN,NaN
2,Angola,AGO,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,96.643701,122.117809,122.242944,143.703204,131.751875,108.680670,95.220799,NaN,NaN,NaN
3,Albania,ALB,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,203.208588,246.803760,246.742546,277.668997,313.262897,264.434603,271.543043,NaN,NaN,NaN
4,Andorra,AND,Current health expenditure per capita (current...,SH.XPD.CHEX.PC.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3754.731346,4013.911834,3857.161116,4107.733984,4346.258747,3698.117574,3834.730581,NaN,NaN,NaN


In [12]:
health_df = health_df.drop(['Country Code','Indicator Name','Indicator Code','1960','1961','1962','1963','1964','1965','1966','1967','1968','1969','1970','1971','1972','1973',
                      '1974','1975','1976','1977','1978','1979','1980','1981','1982','1983','1984','1985','1986','1987',
                      '1988','1989','1990','1991','1992','1993','1994','1995','1996','1997','1998','1999','2016','2017',
                      '2018','2019'], axis=1)

In [13]:
health_df.head()

,Country Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,NaN,NaN,16.249542,17.490737,20.927087,24.446512,28.416662,31.840183,38.700492,42.304510,45.587750,51.553259,52.218506,55.967550,60.112761,60.088813
2,Angola,12.963033,28.854245,28.961365,34.718297,49.526718,53.930701,69.424700,92.452016,135.208309,119.808614,96.643701,122.117809,122.242944,143.703204,131.751875,108.680670
3,Albania,75.531472,81.946417,89.858329,113.583982,151.980517,165.865512,172.795596,216.413135,239.684351,206.944820,203.208588,246.803760,246.742546,277.668997,313.262897,264.434603
4,Andorra,2050.647513,2081.275330,2256.349073,2774.089627,3161.482406,3536.122706,3689.705702,4094.544269,4201.729595,3911.895963,3754.731346,4013.911834,3857.161116,4107.733984,4346.258747,3698.117574


Transforming the data in the health expenditure dataframe to make it consistent with the mortality dataset.

In [14]:
health_df_melted = health_df.melt(id_vars=["Country Name"], 
        var_name="Year", 
        value_name="Health_Expenditure")
health_df_melted.head()

,Country Name,Year,Health_Expenditure
0,Aruba,2000,NaN
1,Afghanistan,2000,NaN
2,Angola,2000,12.963033
3,Albania,2000,75.531472
4,Andorra,2000,2050.647513


Filtering out the dataframe based on the list of countries that we are focusing on

In [15]:
health_df_melted = health_df_melted.loc[health_df_melted['Country Name'].isin(list_of_countries)]
health_df_melted.head()

,Country Name,Year,Health_Expenditure
0,Aruba,2000,NaN
7,Argentina,2000,705.199321
10,Antigua and Barbuda,2000,383.915161
21,"Bahamas, The",2000,1084.292860
24,Belize,2000,132.615056


The above dataframe is now cleaned and is consistent with the mortality dataframe. 

#### Reading the Community Health Workers per 1000 people dataset

In [16]:
community_worker_df = pd.read_csv('Community_Workers.csv')

In [17]:
community_worker_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,"Community health workers (per 1,000 people)",SH.MED.CMHW.P3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,"Community health workers (per 1,000 people)",SH.MED.CMHW.P3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Angola,AGO,"Community health workers (per 1,000 people)",SH.MED.CMHW.P3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,ALB,"Community health workers (per 1,000 people)",SH.MED.CMHW.P3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,AND,"Community health workers (per 1,000 people)",SH.MED.CMHW.P3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Cleaning the dataframe and getting it in the form of health expenditure above

In [18]:
community_worker_df = community_worker_df.drop(['Country Code','Indicator Name','Indicator Code','1960','1961','1962','1963','1964','1965','1966','1967','1968','1969','1970','1971','1972','1973',
                      '1974','1975','1976','1977','1978','1979','1980','1981','1982','1983','1984','1985','1986','1987',
                      '1988','1989','1990','1991','1992','1993','1994','1995','1996','1997','1998','1999','2017',
                      '2018','2019'], axis=1)

In [19]:
community_worker_df.head()

,Country Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
community_worker_df_melted = community_worker_df.melt(id_vars=["Country Name"], 
        var_name="Year", 
        value_name="Community_Workers")
community_worker_df.head()

,Country Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We observe that there are a lot of NaN values here, hence considering Community Health Workers per 100 people may not prove to be an accuracte predictor.

### Reading the Physicians per 1000 people dataset

In [21]:
physician_df = pd.read_csv('Physicians.csv')

In [22]:
physician_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,"Physicians (per 1,000 people)",SH.MED.PHYS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,"Physicians (per 1,000 people)",SH.MED.PHYS.ZS,0.035,NaN,NaN,NaN,NaN,0.063,...,0.2396,0.2553,0.2450,0.2894,0.3039,0.2907,0.2840,NaN,NaN,NaN
2,Angola,AGO,"Physicians (per 1,000 people)",SH.MED.PHYS.ZS,0.067,NaN,NaN,NaN,NaN,0.076,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2149,NaN,NaN
3,Albania,ALB,"Physicians (per 1,000 people)",SH.MED.PHYS.ZS,0.276,NaN,NaN,NaN,NaN,0.481,...,1.2379,1.2225,1.2658,1.2706,NaN,NaN,1.1998,NaN,NaN,NaN
4,Andorra,AND,"Physicians (per 1,000 people)",SH.MED.PHYS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0000,NaN,NaN,NaN,NaN,3.3333,NaN,NaN,NaN,NaN


#### Cleaning the physicians dataset to maintain consistency with WHO dataset

In [23]:
physician_df = physician_df.drop(['Country Code','Indicator Name','Indicator Code','1960','1961','1962','1963','1964','1965','1966','1967','1968','1969','1970','1971','1972','1973',
                      '1974','1975','1976','1977','1978','1979','1980','1981','1982','1983','1984','1985','1986','1987',
                      '1988','1989','1990','1991','1992','1993','1994','2019'], axis=1)

In [24]:
physician_df.head()

,Country Name,1995,1996,1997,1998,1999,2000,2001,2002,2003,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,1.120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,NaN,NaN,0.1100,NaN,NaN,NaN,0.1957,NaN,NaN,...,0.2156,0.2396,0.2553,0.2450,0.2894,0.3039,0.2907,0.2840,NaN,NaN
2,Angola,NaN,NaN,0.0584,NaN,NaN,NaN,NaN,NaN,NaN,...,0.1311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2149,NaN
3,Albania,1.306,1.354,1.2950,1.289,1.282,1.389,NaN,1.305,NaN,...,1.1440,1.2379,1.2225,1.2658,1.2706,NaN,NaN,1.1998,NaN,NaN
4,Andorra,2.231,NaN,2.4350,2.470,2.594,2.549,2.5940,NaN,3.3333,...,3.1120,4.0000,NaN,NaN,NaN,NaN,3.3333,NaN,NaN,NaN


In [25]:
physician_df_melted = physician_df.melt(id_vars=["Country Name"], 
        var_name="Year", 
        value_name="Number_of_Physicians")
physician_df_melted.head()

,Country Name,Year,Number_of_Physicians
0,Aruba,1995,1.120
1,Afghanistan,1995,NaN
2,Angola,1995,NaN
3,Albania,1995,1.306
4,Andorra,1995,2.231


Filtering the physician dataframe based on the list of countries that we're looking into

In [26]:
physician_df_melted = physician_df_melted.loc[physician_df_melted['Country Name'].isin(list_of_countries)]
physician_df_melted.head()

,Country Name,Year,Number_of_Physicians
0,Aruba,1995,1.12
7,Argentina,1995,2.68
10,Antigua and Barbuda,1995,0.76
21,"Bahamas, The",1995,1.49
24,Belize,1995,0.60


The Physician per 1000 people dataframe is now cleaned and consistent with the WHO mortality dataset and can be used a predictor moving forward.

### Week 7

Let us look at other indicators and see if we combine them with our dataframe for our analysis